# Import the necessary libraries

In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

### Load the data into Pandas dataframe

In [82]:
df = pd.read_csv("../archive/combined_emotion.csv")
df.drop_duplicates(inplace = True)
df.head()

,sentence,emotion
0,i just feel really helpless and heavy hearted,fear
1,ive enjoyed being able to slouch about relax a...,sad
2,i gave up my internship with the dmrg and am f...,fear
3,i dont know i feel so lost,sad
4,i am a kindergarten teacher and i am thoroughl...,fear


Drop the columns that are not required for the neural network.

Understand the distribution better.

* Create input and output vectors.
* Process the labels.

In [83]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

X = df["sentence"]
Y = df["emotion"]


# Step 1: Convert string labels to integer labels
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)  # Converts 'joy' → 0, 'sad' → 1, etc.

# Step 2: Convert integer labels to one-hot encoding
Y = to_categorical(Y, num_classes=6)

# Print label mapping (optional)
print(dict(zip(label_encoder.classes_, range(len(label_encoder.classes_)))))


Y

{'anger': 0, 'fear': 1, 'joy': 2, 'love': 3, 'sad': 4, 'suprise': 5}


array([[0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.]])

Split into training and test data.

In [84]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


### Process the data
* Tokenize the data and convert the text to sequences.
* Add padding to ensure that all the sequences have the same shape.
* There are many ways of taking the *max_len* and here an arbitrary length of 150 is chosen.

In [85]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)


In [86]:
sequences_matrix

array([[  0,   0,   0, ..., 103,  85,   4],
       [  0,   0,   0, ...,  90,  13,  12],
       [  0,   0,   0, ..., 100,  58,  47],
       ...,
       [  0,   0,   0, ...,   4,   5,  15],
       [  0,   0,   0, ...,  11,  10,  78],
       [  0,   0,   0, ..., 202,   7, 505]])

### RNN
Define the RNN structure.

In [87]:
emotion_classes = df['emotion'].unique()
def RNN():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(max_words, 50, input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(len(emotion_classes), activation='softmax', name='out_layer')(layer)
    model = Model(inputs=inputs, outputs=layer)
    return model

Call the function and compile the model.

In [88]:
from tensorflow.keras.optimizers import Adam

model = RNN()
model.summary()
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])


c:\Users\medkh\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inputs (InputLayer)             │ (None, 150)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_9 (Embedding)         │ (None, 150, 50)        │        50,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 64)             │        29,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FC1 (Dense)                     │ (None, 256)            │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_11 (Activation)      │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ out_layer (Dense)               │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97,622 (381.34 KB)

 Trainable params: 97,622 (381.34 KB)

 Non-trainable params: 0 (0.00 B)

Fit on the training data.

In [89]:
Y_train

array([[0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.]])

In [91]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,validation_split=0.2)

Epoch 1/10
2081/2081 ━━━━━━━━━━━━━━━━━━━━ 196s 94ms/step - accuracy: 0.7343 - loss: 0.7429 - val_accuracy: 0.8210 - val_loss: 0.4895
Epoch 2/10
2081/2081 ━━━━━━━━━━━━━━━━━━━━ 220s 106ms/step - accuracy: 0.8297 - loss: 0.4898 - val_accuracy: 0.8402 - val_loss: 0.4333
Epoch 3/10
2081/2081 ━━━━━━━━━━━━━━━━━━━━ 161s 77ms/step - accuracy: 0.8426 - loss: 0.4422 - val_accuracy: 0.8441 - val_loss: 0.4127
Epoch 4/10
2081/2081 ━━━━━━━━━━━━━━━━━━━━ 150s 72ms/step - accuracy: 0.8477 - loss: 0.4189 - val_accuracy: 0.8488 - val_loss: 0.4040
Epoch 5/10
2081/2081 ━━━━━━━━━━━━━━━━━━━━ 145s 70ms/step - accuracy: 0.8520 - loss: 0.4028 - val_accuracy: 0.8534 - val_loss: 0.3789
Epoch 6/10
2081/2081 ━━━━━━━━━━━━━━━━━━━━ 151s 73ms/step - accuracy: 0.8565 - loss: 0.3881 - val_accuracy: 0.8583 - val_loss: 0.3644
Epoch 7/10
2081/2081 ━━━━━━━━━━━━━━━━━━━━ 150s 72ms/step - accuracy: 0.8634 - loss: 0.3645 - val_accuracy: 0.8656 - val_loss: 0.3407
Epoch 8/10
2081/2081 ━━━━━━━━━━━━━━━━━━━━ 136s 65ms/step - accuracy:

The model performs well on the validation set and this configuration is chosen as the final model.

Process the test set data.

In [74]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

Evaluate the model on the test set.

In [92]:
accr = model.evaluate(test_sequences_matrix,Y_test)

2601/2601 ━━━━━━━━━━━━━━━━━━━━ 25s 10ms/step - accuracy: 0.8730 - loss: 0.3112


In [78]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.250
  Accuracy: 0.886
